In [ ]:
#Create Python Class Library
#Phase 1: Event Data Acquistions.
    #A. Web Scrape Events Data (Done in R already) +++++++++++++++Complete           
        ## leverage Rpy to run script and import data frame?+++++++++++++++Complete
        ## Events , FRB, CEBS, EBA, IMF, FSB+++++++++++++++Complete
        #FSB events not being scrapped properly. 2017 and 2018 events.
        
        ## Normalize and Combine Events with proper tagging.+++++++++++++++Complete
        ## Provide Event Type out of Categories, Announcement, Results, Schedule, Etc.
        

#Phase 2: Indicies and Identifier Acquistions

    #A. World Returns, Daily, Monthly, Consituents.
        ## Fic Codes, Country Level, 
    #B. Region Level Index Returns
        #Ticker, GVKEY, PERMCO, PERMNO
    #C. Sector Level Index Returns
     #Ticker, GVKEY, PERMCO, PERMNO
    #D. Bank Level Returns
        #Get info for bank participants in Euro, Asia, US, etc.
        #Ticker, GVKEY, PERMCO, PERMNO
        #EBA ~91 banks in 2010
        #US ~41 banks
        
    
#Phase 3: Market Data Acquistion
    #A. Equities
    #B. CDS Spreads
    #C. Bond Returns
    #D. IBES Recommendations
    #E. Options Spreads

#Phase 4: Banking Characteristics Acquistion
    #Capital Ratios
    #PPNR componets
    #Shadow Banking Elements
    #FR-14 elements?

#Phase 5: Event Study Analysis

#Phase 6: Cluster Analysis

#Phase 7: Scenario Based Balance Sheet Projections

#Phase 8: Results Analysis.

    

In [ ]:
#import rpy2.rinterface
#import wrds
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pandas as pd
import numpy as np
# Hide warnings if there are any
import warnings
warnings.filterwarnings('ignore')

pandas2ri.activate()

#%load_ext rpy2.ipython




In [317]:
#%%R -o rss_df_1
#Variables for FSB is Year Range and Policy Areas
def fsb_events(yearrange = "2000:2019", policyareas = '''"non-bank-financial-intermediation", "vulnerabilities-assessments"'''):
    print("Starting FSB Events Web Scrape")
    
    fsb_events = ro.r('''
    if (!require("rvest")) {
      install.packages("rvest")
        }
    library("rvest")
    
    if (!require("devtools")) {
      install.packages("devtools")   
    }

   
    if (!require("feedeR")) {
      devtools::install_github("animalnexus/feedr") 
        }
    library("feedeR")
    
    if (!require("stringr")) {
      install.packages("stringr")   
    }
    library(stringr)


    #FSB Global Shadow Banking Report.
    #Policy Area : shadow-banking was scrubbed from website.
    #Working Policy Areas: non-bank-financial-intermediation, vulnerabilities-assessments

    yearrange = '''+ yearrange +'''
    policyareas = c('''+ policyareas +''')

    rss_df_1 = data.frame()
    ytmp = c()

    for (y in policyareas){

          #print(paste0("Policy Area: ",y))
          #Policy Areas List
          ytmp = c(ytmp,y)

          #Tmp rss iteration for policy area
          rss_df_tmp = data.frame()

      for (i in (yearrange))
          {
          
            for (j in 1:12){  
            url =paste0("http://www.fsb.org/",i,"/",j,"/feed/?policy_area=",y)
            urlrss = feed.extract(url)
            print(url)
            #May need verbose switch
            #print(urlrss$items)

            if(nrow(urlrss$items) > 0) {
              names(urlrss$items)[4] = "category"
              urlrss$items[,4] = y
              #urlrss$items$category = y
              urlrss$items$country = "Global"
              rss_df_tmp = rbind(rss_df_tmp,urlrss$items)
              #print(dim(rss_df_tmp))
            }
            
            }

    
          }
     #Dup Checking and tagging for removal
     rss_df_1 = rbind(rss_df_1,rss_df_tmp)
     #dim(rss_df_1)
     dupidx = duplicated(rss_df_1[,1:3])
     rss_df_1[dupidx,4] = str_c(ytmp, collapse = ", ")
    }

#Dup Removal
    #detach("package:feedeR", unload=TRUE)
    library(dplyr)
    rss_df_1 <- 
    rss_df_1 %>% group_by(title,date) %>% filter(duplicated(title,date) | n()==1)
    
    rss_df_1
    ''')
    #May need to consider timezones
    fsb_events["date"] = pd.to_datetime( fsb_events["date"])
    fsb_events["source"] = "FSB"
    #.dt.strftime('%m/%d/%Y')
    print("Ending FSB Events Web Scrape")

    return(fsb_events)

In [268]:
#Method to Parse EBA Events.
def eba_st_events(yearrange = "2009:2019", urlbase = '''"https://www.eba.europa.eu/risk-analysis-and-data/eu-wide-stress-testing"'''):
#EBA Stress Testing
    print("Starting EBA Stress Test Events Web Scrape")

    eba_events = ro.r('''
    if (!require("rvest")) {
      install.packages("rvest")
        }
    library("rvest")

    EBA_StressTest = data.frame()

    urlbase = '''+ urlbase + '''

    ebayears = '''+ yearrange + '''

    for (i in ebayears){
    
      #Need to implement try catch logic
      
      tmpurl = paste0(urlbase,"/",i)
      #May need verbose switch
      #print(tmpurl)
      if(!is.na(tryCatch( tmpurl %>% as.character() %>% read_html(), error = function(e){NA})))
     {
      
      webpage = read_html(tmpurl)
      #scraptmp = html_nodes(webpage, "dl")
  
      #Short Title
      title = webpage %>% html_nodes('dl') %>% html_node("a") %>% html_text()
      #Description
      desc = webpage %>% html_nodes('dl') %>% html_node("dd") %>% html_text()
      #Date
      #date = as.Date(webpage %>% html_nodes('dl') %>% html_node("dd.TLDate") %>% html_text(), format = "%d/%m/%Y")
      date = webpage %>% html_nodes('dl') %>% html_node("dd.TLDate") %>% html_text()
      #URL
      url = paste0("https://www.eba.europa.eu", webpage %>% html_nodes('dl') %>% html_node("a") %>% html_attr("href"))

      tmpobj = data.frame(title,desc,date,url, country = "Europe", category = "StressTest") 
      if(nrow(tmpobj) > 0) {
              
               EBA_StressTest = rbind.data.frame(EBA_StressTest, tmpobj)
            }
     }
}

    EBA_StressTest
    ''')
    #May need to consider timezones
    eba_events["date"] = pd.to_datetime(eba_events["date"])
    eba_events["source"] = "EBA"
    eba_events["source"][eba_events["title"].str.contains("CEBS")] = "CEBS"
    eba_events["source"][eba_events["title"].str.contains("Results of 2010 EU wide stress testing exercise")] = "CEBS"
    print("Ending EBA Stress Test Events Web Scrape")

    #.dt.strftime('%m/%d/%Y')
    return(eba_events)

In [269]:
#Press Releases and News
def eba_pr_events(urlbase = '''"https://www.eba.europa.eu/news-press/news"'''):
    print("Starting EBA Press Release Events Web Scrape")

    eba_events = ro.r('''
    if (!require("rvest")) {
      install.packages("rvest")
        }
    library("rvest")
    
    
    urlpresssite =  '''+ urlbase + '''
    EBA_PressNews = data.frame()
    webpage = read_html(urlpresssite)


    #Short Title
    title = webpage %>% html_nodes("[class = ListItems]") %>% html_nodes('li') %>% html_node("[class = Title]") %>% html_text()
    #Description
    desc = webpage %>% html_nodes("[class = ListItems]") %>% html_nodes('li') %>% html_node("[class = Info]") %>%  html_node("p") %>% html_text()
    #Date
    #date = as.Date(sapply(strsplit(webpage %>% html_nodes("[class = ListItems]") %>% html_nodes('li') %>% html_node("[class = Info]") %>%  html_node("[class = Date]") %>% html_text(), "-"), head , 1) , format = "%d/%m/%Y")
    date = sapply(strsplit(webpage %>% html_nodes("[class = ListItems]") %>% html_nodes('li') %>% html_node("[class = Info]") %>%  html_node("[class = Date]") %>% html_text(), "-"), head , 1)
    #URL
    
    #May need to be dynamic string
    url = paste0("https://www.eba.europa.eu", webpage %>% html_nodes("[class = ListItems]") %>% html_nodes('li') %>% html_node("[class = Title]") %>% html_attr("href"))

    tmpobj = data.frame(title,desc,date,url, country = "Europe", category = "PressRelease") 

    EBA_PressNews = rbind.data.frame(EBA_PressNews, tmpobj)
    
    ''')
     #May need to consider timezones
    eba_events["date"] = pd.to_datetime(eba_events["date"])
    eba_events["source"] = "EBA"
    eba_events["source"][eba_events["title"].str.contains("CEBS")] = "CEBS"
    eba_events["source"][eba_events["title"].str.contains("Results of 2010 EU wide stress testing exercise")] = "CEBS"
    #.dt.strftime('%m/%d/%Y')
    print("Ending EBA Press Release Events Web Scrape")

    return(eba_events)
    
    


In [270]:
#IMF FSAP
def imf_fsap_events(urlbase = '''"https://www.imf.org/en/Publications/Search?series=IMF+Staff+Country+Reports&when=After&title=Financial+System+Stability+Assessment&series=Global%20Financial%20Stability%20Report%20&series=Technical%20Notes%20and%20Manuals"'''):
    imf_fsap_events = ro.r('''
    if (!require("rvest")) {
          install.packages("rvest")
            }
        library("rvest")
    if (!require("stringr")) {
          install.packages("stringr")   
        }
        library(stringr)
    #if (!require("XML")) {
    #      install.packages("XML")
    #        }
    #library(XML)
    
    url = '''+ urlbase +'''
    #Switch for Verbosity
    #url = "https://www.imf.org/en/Publications/Search?series=IMF+Staff+Country+Reports&when=After&title=Financial+System+Stability+Assessment&series=Global%20Financial%20Stability%20Report%20&series=Technical%20Notes%20and%20Manuals"
    #print(url)
    webpage = read_html(url)
    #Pages variable
    pages = webpage %>% html_nodes('p.pages') %>% html_text()
    pages_clean = gsub("[\r\n]", "", pages[1])
    pages_max = as.numeric(gsub("of ","",str_extract(pages_clean, "of [0-9]+")))

    IMF_FSAP_Events = data.frame()

    for (i in 1:pages_max){

      url_tmp = paste0(url,"&page=",i)
      webpage = read_html(url_tmp)
      Title = str_trim(gsub("Title:","",gsub("\\\s+", " ", str_trim(webpage %>% html_nodes("[class = search-results]") %>% html_nodes('[class = \"result-row pub-row\"]') %>% html_node("h6") %>% html_text()))))

      Country = str_trim(sapply(str_split(Title,":"),head,1))

      Author = str_trim(gsub("Author:","",gsub("\\\s+", " ", str_trim(webpage %>% html_nodes("[class = search-results]") %>% html_nodes('[class = \"result-row pub-row\"]') %>% html_node("p.author") %>% html_text()))))

      Series = str_trim(gsub("Series:","",gsub("\\\s+", " ", str_trim(webpage %>% html_nodes("[class = search-results]") %>% html_nodes('[class = \"result-row pub-row\"]') %>% html_node("p:nth-child(3)") %>% html_text()))))

      Date = as.character(as.Date(str_trim(gsub("Date:","",gsub("\\\s+", " ", str_trim(webpage %>% html_nodes("[class = search-results]") %>% html_nodes('[class = \"result-row pub-row\"]') %>% html_node("p:nth-child(4)") %>% html_text())))), format = "%B %d, %Y"))
      #Date = str_trim(gsub("Date:","",gsub("\\\s+", " ", str_trim(webpage %>% html_nodes("[class = search-results]") %>% html_nodes('[class = \"result-row pub-row\"]') %>% html_node("p:nth-child(4)") %>% html_text()))))
      tmp_df = data.frame(Title,Date,Country, category = "FSSA",Author, Series)
      IMF_FSAP_Events = rbind(IMF_FSAP_Events,tmp_df)
                            }
      IMF_FSAP_Events
      
        ''')
    #May need to consider timezones
    imf_fsap_events["Date"] = pd.to_datetime(imf_fsap_events["Date"])
    imf_fsap_events["source"] = "IMF"
    imf_fsap_events.columns = imf_fsap_events.columns.str.lower()
    #.dt.strftime('%m/%d/%Y')
    return(imf_fsap_events)

In [271]:
def frb_events(urlbase = 'https://www.federalreserve.gov', st_urlbase = 'https://www.federalreserve.gov/supervisionreg/',ccaryearrange = "2011:2018", dfastyearrange = "2013:2018" ):
 
    frb_events = ro.r('''
        if (!require("rvest")) {
          install.packages("rvest")
            }
        library("rvest")
        if (!require("stringr")) {
          install.packages("stringr")   
        }
        library(stringr)
        
        urlbase = 'https://www.federalreserve.gov'
        urlbase2 = 'https://www.federalreserve.gov/supervisionreg/'
        url = 'https://www.federalreserve.gov/supervisionreg/ccar.htm'

        #CCAR
        ccar_yearrange = 2011:2018
        ccarpages = c(paste(urlbase2,"ccar-",ccar_yearrange,".htm",sep = ""))
        #2017 rule
        ccarpages = gsub("ccar-2017.htm","ccar-2017-archive.htm", ccarpages)
        #2018 rule
        ccarpages = gsub("ccar-2018.htm","ccar.htm", ccarpages)
        ccarpages = data.frame(urls = ccarpages, category = "CCAR")
  
        #DFAST  
        dfastyearrange = 2013:2018
        dfastpages = c(paste(urlbase2,"dfast-",dfastyearrange,".htm",sep = ""))
        #2018 rule
        dfastpages = gsub("dfast-2018.htm","dfa-stress-tests.htm", dfastpages)
        dfastpages = data.frame(urls = dfastpages, category = "DFAST")

        #Combine into dataframe
        frbpages = rbind(ccarpages,dfastpages)
        frbpages$urls = as.character(frbpages$urls)
        frbpages$category = as.character(frbpages$category)

        #Get title and urls to scrape
        frb_df = data.frame()
        for( i in 1:nrow(frbpages)) 
                {
                  #Verbose Switch
                  #print(frbpages$urls[i])
                  webpage = read_html(frbpages$urls[i])
                  title = str_trim(webpage %>% html_nodes('[id = article]') %>% html_nodes(xpath = '//*[@id="article"]/ul[1]') %>% html_nodes('a') %>% html_text())
                  #url = paste0(urlbase, webpage %>% html_nodes('[id = article]') %>% html_nodes(xpath = '//*[@id="article"]/ul[1]') %>% html_nodes('a') %>% html_attr("href"))
                  url = webpage %>% html_nodes('[id = article]') %>% html_nodes(xpath = '//*[@id="article"]/ul[1]') %>% html_nodes('a') %>% html_attr("href")
                  frb_df_tmp = data.frame(title,url,category = frbpages$category[i])
                  frb_df = rbind(frb_df,frb_df_tmp)
                }

        frb_df$title = as.character(frb_df$title)
        frb_df$url = as.character(frb_df$url)
        frb_df$category = as.character(frb_df$category)
        frb_df$url[grep("^/newsevents/pressreleases", frb_df$url)] = paste(urlbase,frb_df$url[grep("^/newsevents/pressreleases", frb_df$url)],sep = "")

        #Get Date, Time and Title for Press Releases
        frb_df2 = data.frame()

        for(i in 1:nrow(frb_df))
        {
          #Verbose Switch
          #print(frb_df$title[i])
          #print(frb_df$url[i])
          #print(frb_df$category[i])
  
          webpage1 = read_html(frb_df$url[i])

      releasedate = webpage1 %>% html_nodes('[id = article]') %>% html_nodes('p.article__time') %>% html_text()
      title = webpage1 %>% html_nodes('[id = article]') %>% html_nodes('h3.title') %>% html_text()
      releaseTime = str_trim(gsub("For release at", "", webpage1 %>% html_nodes('[id = article]') %>% html_nodes('p.releaseTime') %>% html_text()))
      lastUpdate = str_trim(gsub("Last Update:","",gsub("\\\s+", " ", str_trim(webpage1 %>% html_nodes('[id = lastUpdate]') %>%  html_text()))))
  
      frb_df2_tmp = data.frame(title,releasedate,releaseTime,lastUpdate, url = frb_df$url[i], country = "USA", category = frb_df$category[i])
      frb_df2 = rbind(frb_df2, frb_df2_tmp)
            }

        frb_df2
    ''')
    frb_events['date1'] = frb_events["releasedate"].astype(str) + " " + frb_events["releaseTime"].astype(str).replace("For immediate release","9:00 a.m. EST")
    frb_events['date1'] = frb_events['date1'].str.replace("a.m.","AM").str.replace("p.m.","PM") 
    frb_events['date'] = pd.to_datetime(frb_events['date1'], utc = True).dt.tz_convert("UTC").dt.tz_convert("US/Eastern")
    frb_events['source'] = "FRB"
    return(frb_events)

In [322]:
#Wrapper Function to get events
def getevents_data(eventscolumnlist = ['title','date','country','category','source']
                   , source = ["fsb","eba","imf","frb"]):
    #Make switched and arguements to select events to gather and return
    events_combined = pd.DataFrame({'empty' : []})
    tmp_raw = []
    if "fsb" in source:
        print("Started: Scraping Events from FSB")
        tmp_raw = fsb_events()
        print("Finshed: Scraping Events from FSB")
        print("Adding Events to ResultsDataframe")
        if not events_combined.empty:
            events_combined = pd.concat([events_combined,tmp_raw[eventscolumnlist]])
        else:
            events_combined = tmp_raw[eventscolumnlist]
            
    if "eba" in source:                                
        print("Started: Scraping Events from EBA")
        print("Started: Scraping Stress Test Events from EBA")
        tmp_raw = eba_st_events()
        print("Finished: Scraping Stress Test Events from EBA")
        print("Adding Events to ResultsDataframe")
        if not events_combined.empty:
            events_combined = pd.concat([events_combined,tmp_raw[eventscolumnlist]])
        else:
            events_combined = tmp_raw[eventscolumnlist]
        
        print("Started: Scraping Press Release Events from EBA")
        tmp_raw = eba_pr_events()
        print("Finished: Scraping Press Release Events from EBA")
        print("Adding Events to ResultsDataframe")
        
        if not events_combined.empty:
            events_combined = pd.concat([events_combined,tmp_raw[eventscolumnlist]])
        else:
            events_combined = tmp_raw[eventscolumnlist]
                                
    if "imf" in source:
                                
        print("Started: Scraping FSAP Events from IMF")
        tmp_raw = imf_fsap_events()
        print("Finished: Scraping FSAP Events from IMF")
        print("Adding Events to ResultsDataframe")
        if not events_combined.empty:
            events_combined = pd.concat([events_combined,tmp_raw[eventscolumnlist]])
        else:
            events_combined = tmp_raw[eventscolumnlist]
    if "frb" in source:    
        print("Started: Scraping Stress Test Press Releases from FRB")
        tmp_raw = frb_events()
        print("Finished: Scraping Stress Test Press Releases from FRB")
        print("Adding Events to ResultsDataframe")
        if not events_combined.empty:
            events_combined = pd.concat([events_combined,tmp_raw[eventscolumnlist]])
        else:
            events_combined = tmp_raw[eventscolumnlist]
    
    print("Events Scraping Completed")                                
    return(events_combined)

In [324]:
#Tagg the event types
#FRB
#Results
events[events['title'].str.contains("announces results|releases results")]

,title,date,country,category,source
9,Federal Reserve announces results of Comprehen...,2013-03-14 09:00:00-04:00,USA,CCAR,FRB
16,Federal Reserve releases results of Comprehens...,2014-03-26 09:00:00-04:00,USA,CCAR,FRB
20,Federal Reserve releases results of Comprehens...,2015-03-11 16:30:00-04:00,USA,CCAR,FRB
23,Federal Reserve releases results of Comprehens...,2016-06-29 16:30:00-04:00,USA,CCAR,FRB
27,Federal Reserve releases results of Comprehens...,2017-06-28 16:30:00-04:00,USA,CCAR,FRB
33,Federal Reserve releases results of Comprehens...,2018-06-28 16:30:00-04:00,USA,CCAR,FRB
47,Federal Reserve releases results of supervisor...,2015-03-05 16:30:00-05:00,USA,DFAST,FRB
49,Federal Reserve releases results of supervisor...,2016-06-23 16:30:00-04:00,USA,DFAST,FRB
51,Federal Reserve Board releases results of supe...,2017-06-22 16:30:00-04:00,USA,DFAST,FRB
53,Federal Reserve Board releases results of supe...,2018-06-21 16:30:00-04:00,USA,DFAST,FRB


In [326]:
#FSB
#Report Release
events[events['title'].str.contains("Global Shadow Banking")]

,title,date,country,category,source
89,Global Shadow Banking Monitoring Report 2017,2018-03-05 06:00:55+00:00,Global,non-bank-financial-intermediation,FSB
92,FSB publishes Global Shadow Banking Monitoring...,2018-03-05 03:00:13+00:00,Global,non-bank-financial-intermediation,FSB
128,Global Shadow Banking Monitoring Report 2012,2012-11-18 02:50:00+00:00,Global,"non-bank-financial-intermediation, vulnerabili...",FSB
130,The FSB publishes its 2013 Global Shadow Banki...,2013-11-14 06:36:00+00:00,Global,"non-bank-financial-intermediation, vulnerabili...",FSB
131,Global Shadow Banking Monitoring Report 2013,2013-11-14 06:08:00+00:00,Global,"non-bank-financial-intermediation, vulnerabili...",FSB
137,The FSB publishes its 2014 Global Shadow Banki...,2014-11-04 10:45:19+00:00,Global,"non-bank-financial-intermediation, vulnerabili...",FSB
138,Global Shadow Banking Monitoring Report 2014,2014-11-04 10:39:37+00:00,Global,"non-bank-financial-intermediation, vulnerabili...",FSB
145,Global Shadow Banking Monitoring Report 2015,2015-11-12 07:04:06+00:00,Global,"non-bank-financial-intermediation, vulnerabili...",FSB
146,Global Shadow Banking Monitoring Report 2015 -...,2015-11-12 07:03:54+00:00,Global,"non-bank-financial-intermediation, vulnerabili...",FSB
147,Global Shadow Banking Monitoring Report 2015 -...,2015-11-12 07:02:04+00:00,Global,"non-bank-financial-intermediation, vulnerabili...",FSB


In [329]:
#EBA
#Results
events[events['title'].str.contains("stress test results")]

,title,date,country,category,source
12,EBA publishes 2014 EU-wide stress test results,2014-10-26 00:00:00,Europe,StressTest,EBA
19,EBA publishes 2016 EU-wide stress test results,2016-07-29 00:00:00,Europe,StressTest,EBA
22,EBA publishes 2018 EU-wide stress test results,2018-02-11 00:00:00,Europe,StressTest,EBA
23,EBA publishes 2018 EU-wide stress test results,2018-02-11 00:00:00,Europe,PressRelease,EBA
353,EBA publishes 2016 EU-wide stress test results,2016-07-29 00:00:00,Europe,PressRelease,EBA
356,EBA discloses example of templates for stress ...,2016-07-27 00:00:00,Europe,PressRelease,EBA
364,EBA announces timing for publication of 2016 E...,2016-07-19 00:00:00,Europe,PressRelease,EBA
374,EBA clarifies use of 2016 EU-wide stress test ...,2016-01-07 00:00:00,Europe,PressRelease,EBA
650,Revised RBS 2014-EU wide stress test results,2014-11-21 00:00:00,Europe,PressRelease,EBA
666,EBA publishes 2014 EU-wide stress test results,2014-10-26 00:00:00,Europe,PressRelease,EBA


In [330]:
#IMF 
#Results
events[events['source'] == "IMF"]

,title,date,country,category,source
0,Jamaica : Financial System Stability Assessment,2018-12-03 00:00:00,Jamaica,FSSA,IMF
1,Brazil : Financial System Stability Assessment,2018-11-30 00:00:00,Brazil,FSSA,IMF
2,Peru : Financial System Stability Assessment,2018-07-25 00:00:00,Peru,FSSA,IMF
3,Euro Area Policies : Financial System Stabilit...,2018-07-19 00:00:00,Euro Area Policies,FSSA,IMF
4,Namibia : Financial System Stability Assessment,2018-03-15 00:00:00,Namibia,FSSA,IMF
5,Belgium : Financial System Stability Assessment,2018-03-08 00:00:00,Belgium,FSSA,IMF
6,Belgium : Financial System Stability Assessmen...,2018-03-08 00:00:00,Belgium,FSSA,IMF
7,Belgium : Financial System Stability Assessmen...,2018-03-08 00:00:00,Belgium,FSSA,IMF
8,Belgium : Financial System Stability Assessmen...,2018-03-08 00:00:00,Belgium,FSSA,IMF
9,India : Financial System Stability Assessment-...,2017-12-21 00:00:00,India,FSSA,IMF


In [321]:
#Insertition Point
events = []
events = getevents_data()




Started: Scraping Events from FSB
Starting FSB Events Web Scrape
[1]
 "http://www.fsb.org/2000/1/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/2/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/3/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/4/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/5/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/6/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/7/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/8/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/9/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/10/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2000/11/feed/?policy_area=non-bank-fina

[1]
 "http://www.fsb.org/2008/1/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/2/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/3/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/4/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/5/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/6/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/7/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/8/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/9/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/10/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/11/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2008/12/feed/?po

[1]
 "http://www.fsb.org/2016/6/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2016/7/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2016/8/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2016/9/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2016/10/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2016/11/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2016/12/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2017/1/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2017/2/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2017/3/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2017/4/feed/?policy_area=non-bank-financial-intermediation"


[1]
 "http://www.fsb.org/2017/5/feed/?po

[1]
 "http://www.fsb.org/2004/10/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2004/11/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2004/12/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2005/1/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2005/2/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2005/3/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2005/4/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2005/5/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2005/6/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2005/7/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2005/8/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2005/9/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/

[1]
 "http://www.fsb.org/2013/7/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2013/8/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2013/9/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2013/10/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2013/11/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2013/12/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2014/1/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2014/2/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2014/3/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2014/4/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2014/5/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/2014/6/feed/?policy_area=vulnerabilities-assessments"


[1]
 "http://www.fsb.org/